In [0]:
Personal Website - https://vitthal-bhandari.github.io/
GitHub Link - https://github.com/vitthal-bhandari


In [1]:
!nvidia-smi

In [2]:
from platform import python_version
print(python_version())

3.10.11


In [3]:
import torch
device = torch.device('cuda:0')

In [5]:
import torch

print(torch.cuda.is_available())
print(torch.__version__)


True
2.0.1


In [6]:
pip install -U openai-whisper

In [7]:
import pandas as pd
import numpy as np
import requests
import torch
import whisper
import json
import os

In [16]:
# reading a subset of the dataset

data_subset = pd.read_csv('podcast_data_2023-07-26.csv')
data_subset = data_subset.sample(frac=1).reset_index(drop=True)
data = data_subset.head(10)
data

,Episode ID,Show Name,Full Date,Title,Description,Partisan Leaning,Category on Apple,Entered Dataset,Audio URL
0,204,AMERICA OUT LOUD PODCAST NETWORK,2023-07-16,the end game connecting the dots of covid shot...,truth for health with drlee4america – everythi...,More Conservative,News,Apple Recommendation,https://media.blubrry.com/truthforhealth/media...
1,6265,Conservative Review with Daniel Horowitz,2023-03-14,we continue to fail with red-state legislature...,without a sense of direction and purpose on po...,More Conservative,News,Apple Recommendation,https://www.podtrac.com/pts/redirect.mp3/pdst....
2,6214,Conservative Review with Daniel Horowitz,2023-06-01,"pandemic of mental contagion, autism, mass hyp...","today, we cover some of the most vexing issues...",More Conservative,News,Apple Recommendation,https://www.podtrac.com/pts/redirect.mp3/pdst....
3,1519,Bannon's War Room,2022-08-30,episode 2117: alex jones: the great reset and ...,episode 2117: alex jones: the great reset and ...,More Conservative,News,Apple Top 100,https://chrt.fm/track/5B4639/storage.warroom.o...
4,438,American Thought Leaders,2023-01-26,alvin lui: how schools are weaponizing ‘inclus...,“my great grandfather ran from communism. the ...,More Conservative,News,Apple Recommendation,https://podcasts.captivate.fm/media/d8b788fe-a...
5,5472,Candace Owens,2023-03-29,ep. 124 - interview with vivek ramaswamy part 2,"2024 presidential candidate, vivek ramaswamy, ...",More Conservative,News,Apple Top 100,https://dts.podtrac.com/redirect.mp3/chrt.fm/t...
6,6350,Conservative Review with Daniel Horowitz,2022-10-24,how republicans offer the illusion of an oppos...,how do we make the election win of republicans...,More Conservative,News,Apple Recommendation,https://www.podtrac.com/pts/redirect.mp3/pdst....
7,6351,Conservative Review with Daniel Horowitz,2022-10-21,how to pressure the gop to actually make the e...,the election mandate will not fulfill itself. ...,More Conservative,News,Apple Recommendation,https://www.podtrac.com/pts/redirect.mp3/pdst....
8,5442,Candace Owens,2023-05-11,why chelsea handler loves dylan mulvaney,why transgenderism is popular among radical fe...,More Conservative,News,Apple Top 100,https://dts.podtrac.com/redirect.mp3/chrt.fm/t...
9,6355,Conservative Review with Daniel Horowitz,2022-10-14,republicans aren’t even campaigning on the iss...,not only are republicans not confronting democ...,More Conservative,News,Apple Recommendation,https://www.podtrac.com/pts/redirect.mp3/pdst....


In [14]:
data["Audio URL"][0]

'https://www.podtrac.com/pts/redirect.mp3/pdst.fm/e/pfx.vpixl.com/4V6da/arttrk.com/p/BLZM1/chtbl.com/track/384GD8/verifi.podscribe.com/rss/p/traffic.megaphone.fm/BMDC5125488112.mp3?updated=1668707287'

In [15]:
model = whisper.load_model('base.en', device="cuda:0")

In [11]:
# function to transribe audio using whisper
def transcribe_audio_url(episode_id, show_name, full_date, url):
    
    # Download the audio file from the URL
    response = requests.get(url)
    
    temp_file = str(episode_id) + "_" + str(show_name) + "_" + str(full_date) + ".mp3"
    
    filepath = os.path.join('audios', temp_file)
    
    with open(filepath, "wb") as file:
        file.write(response.content)
    
    transcription = model.transcribe(filepath)
    
    # Serializing json
    json_object = json.dumps(transcription, indent=4, sort_keys=True)
    
    transcription_filename = str(episode_id) + "_" + str(show_name) + "_" + str(full_date) + "_transcription.json"
    
    transcription_filepath = os.path.join('transcriptions', transcription_filename)
    
    # Writing to categorization_tweets.json
    with open(transcription_filepath, "w") as outfile:
        outfile.write(json_object)
    
    print(f"Successfully transcribed episode ID {episode_id} from show '{show_name}'.")

In [20]:
import time

start_time = time.time()

# iterating through the dataframe
for index, row in new_data.iterrows():
    episode_id = row["Episode ID"]
    show_name = row["Show Name"]
    full_date = row["Full Date"]
    url = row["Audio URL"]
    try:
        transcribe_audio_url(episode_id, show_name, full_date, url)
    except Exception as e:
        print(e)

end_time = time.time()
execution_time = end_time - start_time
minutes = execution_time / 60

print(f"Successfully transcribed all apisodes in {minutes} minutes.")

Successfully transcribed episode ID 438 from show 'American Thought Leaders'.
Successfully transcribed episode ID 5472 from show 'Candace Owens'.
Successfully transcribed episode ID 6350 from show 'Conservative Review with Daniel Horowitz'.
Successfully transcribed episode ID 6351 from show 'Conservative Review with Daniel Horowitz'.
Successfully transcribed episode ID 5442 from show 'Candace Owens'.
Successfully transcribed episode ID 6355 from show 'Conservative Review with Daniel Horowitz'.
Successfully transcribed all apisodes in 33.70048033793767 minutes.
